In [16]:
import os
import shutil
import requests
import ipykernel
import json
from notebook import notebookapp
from IPython.display import display, Javascript

def save_script(target_dir, name):
    # Attempt to get the notebook path if running inside Jupyter
    try:
        # This line will throw a NameError if not running in a Jupyter environment
        connection_file = os.path.basename(ipykernel.get_connection_file())
        kernel_id = connection_file.split('-', 1)[1].split('.')[0]

        found = False
        for srv in notebookapp.list_running_servers():
            try:
                response = requests.get(f'{srv["url"]}api/sessions',
                                        headers={'Authorization': f'token {srv["token"]}'},
                                        verify=False)  # It's not recommended to skip verification in production
                response.raise_for_status()
                sessions = json.loads(response.text)
                for sess in sessions:
                    if sess['kernel']['id'] == kernel_id:
                        notebook_path = sess['notebook']['path']
                        full_path = os.path.join(srv['notebook_dir'], notebook_path)
                        target_path = os.path.join(target_dir, f'{name}.ipynb')
                        shutil.copy2(full_path, target_path)
                        print(f'Notebook saved as {target_path}')
                        found = True
                        break
            except Exception as e:
                raise Exception(f'Error while trying to save the notebook: {e}')
            if found:
                break
        if not found:
            raise Exception('Could not find the notebook path. Are you running this in a Jupyter environment?')

    except NameError:
        # If we're not in a Jupyter environment, it's likely a standard Python script
        script_path = os.path.abspath('')
        full_path = os.path.join(script_path, __file__)
        target_path = os.path.join(target_dir, f'{name}.py')
        shutil.copy2(full_path, target_path)
        print(f'Script saved as {target_path}')

    except Exception as e:
        # If any other exception occurred, we re-raise it
        raise Exception(f'An error occurred: {e}')

save_script("./", "trial")

Exception: An error occurred: Could not find the notebook path. Are you running this in a Jupyter environment?